In [1]:
# Example taken from https://docs.fast.ai/tabular.html#Defining-a-model
from fastai.tabular import * 
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [2]:
procs = [FillMissing, Categorify, Normalize]
valid_idx = range(len(df)-2000, len(df))
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

learn = tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

['capital-gain', 'age', 'fnlwgt', 'capital-loss', 'hours-per-week', 'education-num']


epoch,train_loss,valid_loss,accuracy,time
0,0.331154,0.320145,0.847500,00:03


In [38]:
# Local predictions
learn.predict(df.iloc[0])

(Category >=50k, tensor(1), tensor([0.2772, 0.7228]))

In [24]:
import blazee
import os
import logging
logging.basicConfig(level=logging.INFO)
%load_ext autoreload
%autoreload 2

# Deploy the model on Blazee
api_key = os.environ['BLAZEE_API_KEY']
bz = blazee.Blazee(api_key)
[m.delete() for m in bz.all_models() if m.name == 'FastAi Adult']
model = bz.deploy_model(learn, model_name="FastAi Adult")
model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:root:Uploading model version to Blazee  (180.5 KB)...
INFO:root:Deploying new model version: v1...
INFO:root:Successfully deployed model version 4b7ba023-4aac-4f5c-be7e-b922a96f129b


<BlazeeModel 'FastAi Adult'
	id=859f4cd8-ebd9-4000-8578-5482ab0dfd7b>

In [48]:
sample_data = df.iloc[0].to_dict()
print(sample_data)
model.predict(sample_data)

{'age': 49, 'workclass': ' Private', 'fnlwgt': 101320, 'education': ' Assoc-acdm', 'education-num': 12.0, 'marital-status': ' Married-civ-spouse', 'occupation': nan, 'relationship': ' Wife', 'race': ' White', 'sex': ' Female', 'capital-gain': 0, 'capital-loss': 1902, 'hours-per-week': 40, 'native-country': ' United-States', 'salary': '>=50k'}


<Prediction
	prediction=1
	probas=[0.2772163152694702, 0.7227836847305298]>